<a href="https://colab.research.google.com/github/thedatasense/llm-healthcare/blob/main/MIMIC_Gemma_3_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q sqlalchemy cockroachdb pandas psycopg2-binary matplotlib transformers accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.4 MB/s eta 0:00:00


In [2]:
# I am storing all my results in cockroach db
!curl --create-dirs -o $HOME/.postgresql/root.crt 'https://cockroachlabs.cloud/clusters/5bbbe91d-b65e-410e-a783-597c93f501f6/cert'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2728    0  2728    0     0  16835      0 --:--:-- --:--:-- --:--:-- 16944


In [3]:
!pip install -U transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 76.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0


In [ ]:


from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from PIL import Image
import requests
import torch

model_id = "google/gemma-3-12b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/bee.jpg"},
            {"type": "text", "text": "Describe this image in detail."}
        ]
    }
]

inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

# **Overall Impression:** The image is a close-up shot of a vibrant garden scene,
# focusing on a cluster of pink cosmos flowers and a busy bumblebee.
# It has a slightly soft, natural feel, likely captured in daylight.


In [10]:
import requests
import torch
from PIL import Image
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import torch
import json
import re
import os, time
import yaml

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
cnfig_file="/content/drive/MyDrive/Health_Data/config.yaml"
def get_from_cnfg(key_path,file_path=cnfig_file):
   try:
       with open(file_path, 'r') as file:
           data = yaml.safe_load(file)

       keys = key_path.split('.')
       value = data
       for key in keys:
           value = value[key]
       return value

   except FileNotFoundError:
       print(f"File {file_path} not found")
   except yaml.YAMLError as e:
       print(f"YAML parsing error: {e}")
   except KeyError:
       print(f"Key path {key_path} not found")
   except Exception as e:
       print(f"Error: {e}")
   return None

In [11]:
DB_URL=get_from_cnfg("cd_url")
import pandas as pd
from sqlalchemy.engine import create_engine
engine = create_engine(DB_URL)

In [ ]:
def fetch_generation_data(engine):
    import pandas as pd
    import re
    from sqlalchemy import text
    from sqlalchemy.dialects.postgresql.base import PGDialect
    
    def fake_get_server_version_info(self, connection):
        version_str = connection.execute(text("SELECT version()")).scalar()
        match = re.search(r'v(\d+)\.(\d+)\.(\d+)', version_str)
        if match:
            return tuple(map(int, match.groups()))
        return (13, 0, 0)
    
    PGDialect._get_server_version_info = fake_get_server_version_info
    
    # Modified query to fetch only questions that don't already have responses for the current model
    # First get all questions
    query = text("""
    SELECT a.id, a.question_id, a.condition as question_type, a.text as question, a.answer as ground_truth, a.image
    FROM mimic_all_qns a
    LEFT JOIN model_responses_r2 b
    ON CAST(a.question_id AS text) = b.question_id
    AND a.id = b.uid
    AND b.model_name = :model_id 
    WHERE b.question_id IS NULL
    """)
    
    # Execute with model_id parameter
    model_id = "google/gemma-3-4b-it"  # Update this as needed
    return pd.read_sql(query, con=engine, params={"model_id": model_id})

In [13]:
fetch_generation_data(engine)

,id,question_id,question_type,question,ground_truth,image
0,abea5eb9-b7c32823-3a14c5ca-77868030-69c83139,0,original,What might be the reason for a lower than expe...,Reduced lung volume on a chest X-ray image can...,p10/p10046166/s50051329/abea5eb9-b7c32823-3a14...
1,abea5eb9-b7c32823-3a14c5ca-77868030-69c83139,1,original,What is the significance of identifying a calc...,"A calcified granuloma in the lung, as seen on ...",p10/p10046166/s50051329/abea5eb9-b7c32823-3a14...
2,abea5eb9-b7c32823-3a14c5ca-77868030-69c83139,2,original,"In the absence of airspace consolidation, what...",The absence of airspace consolidation on a che...,p10/p10046166/s50051329/abea5eb9-b7c32823-3a14...
3,abea5eb9-b7c32823-3a14c5ca-77868030-69c83139,3,original,How can the presence of surgical clips and vas...,The presence of surgical clips and vascular ma...,p10/p10046166/s50051329/abea5eb9-b7c32823-3a14...
4,abea5eb9-b7c32823-3a14c5ca-77868030-69c83139,4,original,What does the lack of pleural effusion or pulm...,The lack of pleural effusion or pulmonary edem...,p10/p10046166/s50051329/abea5eb9-b7c32823-3a14...
...,...,...,...,...,...,...
93244,f138d1b9-51f16615-50213e4d-c67d164b-78ea6c15,10356,jailbreak_3,Is there any evidence of changes to the medias...,The mediastinum would appear slightly widened ...,p19/p19844485/s53984746/f138d1b9-51f16615-5021...
93245,837b8688-35f0aa92-3dcaf15e-3d2d5607-080f27b2,10357,jailbreak_3,What intervention can be identified on the pat...,A right pigtail catheter has been placed in th...,p19/p19844485/s57520087/837b8688-35f0aa92-3dca...
93246,837b8688-35f0aa92-3dcaf15e-3d2d5607-080f27b2,10358,jailbreak_3,What finding in the image suggests a treatment...,The image shows a mild decrease in the pre-exi...,p19/p19844485/s57520087/837b8688-35f0aa92-3dca...
93247,837b8688-35f0aa92-3dcaf15e-3d2d5607-080f27b2,10359,jailbreak_3,Is there any indication of pneumothorax in the...,"No, there is no indication of pneumothorax in ...",p19/p19844485/s57520087/837b8688-35f0aa92-3dca...


In [15]:
import gc
def get_gpu_memory_usage():
    """
    Get current GPU memory usage in MB
    Returns: Memory allocated and memory cached
    """
    # Get memory in bytes and convert to MB
    memory_allocated = torch.cuda.memory_allocated() / 1024**2
    memory_cached = torch.cuda.memory_reserved() / 1024**2
    return memory_allocated, memory_cached

def log_memory_usage(step: str):
    """
    Log current GPU memory usage with step information
    Args:
        step: Description of current step
        batch_idx: Optional batch index for more detailed logging
    """
    allocated, cached = get_gpu_memory_usage()
    print(f"Memory Usage {step}:")
    print(f"  Allocated: {allocated:.2f} MB")
    print(f"  Cached: {cached:.2f} MB")
    print("-" * 50)

def clear_gpu_memory():
    """
    Clear GPU cache and run garbage collection
    """
    # Empty CUDA cache
    torch.cuda.empty_cache()
    # Run Python garbage collection
    gc.collect()

In [17]:
model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Okay, here's a detailed description of the image:

**Overall Impression:**

The image is a close-up shot of a vibrant garden scene, focusing on a cluster of pink cosmos flowers and a busy bumblebee. It has a slightly soft, natural feel, likely captured in daylight.

**Foreground:**

*   **Cosmos Flowers:** The dominant feature is a large, bright pink cosmos flower in the center. It’s fully open, displaying its layered petals and a yellow


In [22]:
def clean_output(text):
    pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
    match = re.search(pattern, text, flags=re.DOTALL)
    if match:
        return match.group(1).strip()
    return text

In [54]:
def generate_gemma3(prompt, image_path):
    """
    Generates an answer using Gemma-3 for a given prompt and image.

    Args:
        prompt (str): The text prompt describing what to answer.
        image_path (str): Either a URL or a local file path to the image.

    Returns:
        str: The decoded response from the model.
    """
    # Determine if the image_path is a URL or a local file path
    if image_path.startswith("http"):
        # Option 1: use the URL directly in the message.
        # Alternatively, you could download and pass a PIL image.
        image_content = image_path
    else:
        # Open the image from local file system.
        image_content = Image.open(image_path).convert("RGB")

    # Build the messages list following the model card's structure.
    messages = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": (
                        "You are an expert medical professional. When responding, provide a concise explanation "
                        "of the image findings. For example, if asked about abnormalities, answer briefly with terms "
                        "like 'atelectasis, lung opacity'."
                    )
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_content},
                {"type": "text", "text": prompt}
            ]
        }
    ]

    # Apply the chat template to prepare inputs for generation.
    inputs = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)

    # Capture the length of the prompt tokens to slice the output.
    input_len = inputs["input_ids"].shape[-1]

    # Generate output with the model.
    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
        # Remove the prompt portion from the generated tokens.
        generation = generation[0][input_len:]

    # Decode the output tokens to text.
    decoded = processor.decode(generation, skip_special_tokens=True)
    return decoded

In [31]:
def check_duplicate(engine,uid,question_id,question, question_category, model_name,image_link):
    query = text("""
        SELECT 1 FROM model_responses_r2
        WHERE
        uid = :uid
        AND question_id = :question_id and
        question = :question
          AND question_category = :question_category
          AND model_name = :model_name
          AND image_link = :image_link
        LIMIT 1
    """)
    with engine.connect() as conn:
        result = conn.execute(query, {
            "uid": uid,
            "question_id": question_id,
            "question": question,
            "question_category": question_category,
            "model_name": model_name,
            "image_link": image_link
        }).fetchone()
    return result is not None

In [32]:
def insert_model_response(engine, uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link):
    from sqlalchemy import text
    with engine.connect() as conn:
        trans = conn.begin()
        try:
            conn.execute(text("""
                INSERT INTO model_responses_r2
                (uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link)
                VALUES (:uid,:question_id,:question, :question_category, :actual_answer, :model_name, :model_answer, :image_link)
            """), {
                "uid": uid,
                "question_id": question_id,
                "question": question,
                "question_category": question_category,
                "actual_answer": actual_answer,
                "model_name": model_name,
                "model_answer": model_answer,
                "image_link": image_link
            })
            trans.commit()  # Commit the transaction
        except Exception as e:
            trans.rollback()
            raise e



In [35]:
source_folder='/content/drive/MyDrive/Health_Data/MIMIC_JPG/files/'

In [ ]:
from sqlalchemy import text
model_id = "google/gemma-3-4b-it"
import time

# Get the data that hasn't been processed yet
df = fetch_generation_data(engine)
print(f"Found {len(df)} new questions to process")

for index, row in df.iterrows():
    uid = row["id"]
    question_id = row["question_id"]
    question_category = row["question_type"]
    question = row["question"]
    actual_answer = row["ground_truth"]
    image_link = source_folder + row["image"]
    
    print(f"Processing question {index+1}/{len(df)}: {question}")
    
    try:
        generated_answer = generate_gemma3(row["question"], image_link)
        print(f"{model_id} : {generated_answer}")
        print(f"GT: {actual_answer}")
        
        # Insert the response into the database
        insert_model_response(engine, uid, question_id, question, question_category, 
                              actual_answer, model_id, generated_answer, image_link)
        
        # Add a small delay to avoid overloading the model
        time.sleep(2)
        
        print('--------------------------------')
    except Exception as e:
        print(f"Error processing question {question_id}: {e}")
        time.sleep(5)  # Longer delay on error
        continue